## Imports

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
from datetime import datetime, timedelta

## Constants

In [80]:
# API Key
PersonalApiKey = "JbMPn9bSpFPNS7Z7PcZy"
# Pairs
pairs1 = {"W": ['OWF/NYM_RB_RB', 1.0], "X": ['OWF/ICE_G_G', 0.0033]}
pairs2 = {"Y": ['OWF/ICE_B_B', 1.0], "Z": ['OWF/ICE_G_G', 0.1147]}
# start and stop dates
start_date = "2020-12-03"
end_date = "2022-08-31"

In [81]:
# possible quarters
class quarters:
    def __init__(self, start_date):
        self.quarters = ['H', 'M', 'U', 'Z']
        self.year = int(start_date.split("-")[0])
        month = int(start_date.split("-")[1])
        if month < 3:
            self.index = 0
        elif month < 6:
            self.index = 1
        elif month < 9:
            self.index = 2
        elif month < 12:
            self.index = 3
        else:
            self.year += 1
            self.index = 0

        self.label = "_" + self.quarters[self.index] + str(self.year) + "_IVM"

    def next(self):
        if self.index == 3:
            self.year += 1
            self.index = 0
        else:
            self.index += 1

        self.label = "_" + self.quarters[self.index] + str(self.year) + "_IVM"

## Functions

In [82]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
@functools.lru_cache(maxsize=1600)
def fetch_quandl(my_data_items, trim_start=None, trim_end=None):
    qdata = quandl.get(list(my_data_items), returns="pandas",
                      trim_start=trim_start, trim_end=trim_end,
                      api_key=PersonalApiKey)
    return qdata

In [83]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
and was modified for futures contract analysis
"""
def clean_quandl_future_columns(dataframe, pair):
    replacement_columns = {}
    for c in dataframe.columns:
        series_name, variable = c.split(' - ')
        source_name, asset = series_name.split('/')
        replacement_columns[c] = pair + "_" + variable
    renamed_data = dataframe.rename(columns=replacement_columns)
    return renamed_data[[pair + "_" + 'Future', pair + "_" + 'DtT']]

In [84]:
def fetch_second_month_futures(contract_label, pair, start_date, end_date):
    data= clean_quandl_future_columns(fetch_quandl((contract_label,),
                                                   start_date, end_date),
                                      pair)

    return data[data[pair + "_" + 'DtT'] > 30][pair + "_" + 'Future']
    #only returning future prices that expires in more than 30 days

In [111]:
def compile_data(pairs, start_date, end_date):
    ret_list = []
    for pair, detail in pairs.items():
        df = pd.DataFrame(dtype = float)
        start = start_date
        heading = detail[0]
        multiplier = detail[1]
        tail = quarters(start_date)
        while(start < end_date):
            temp = fetch_second_month_futures(heading + tail.label, pair, start, end_date)
            df = pd.concat([df, temp])
            start = (temp.index[-1] + timedelta(days = 1)).strftime("%Y-%m-%d")
            tail.next()
        df = df.rename(columns = {0:pair + '_Future'})
        df[pair + '_Future'] *= multiplier
        ret_list.append(df)
    return pd.concat(ret_list, axis = 1, join = 'inner')

In [114]:
df = compile_data(pairs1, start_date, end_date)
df['Spread'] = df['X_Future'] - df['W_Future']
df

,W_Future,X_Future,Spread
2020-12-03,1.2867,1.335675,0.048975
2020-12-04,1.2951,1.339800,0.044700
2020-12-07,1.2850,1.353000,0.068000
2020-12-08,1.2851,1.352175,0.067075
2020-12-09,1.2983,1.342275,0.043975
...,...,...,...
2022-08-25,2.5303,3.566475,1.036175
2022-08-26,2.5483,3.573900,1.025600
2022-08-29,2.6100,3.569775,0.959775
2022-08-30,2.4482,3.366825,0.918625


## Introduction

True

## Data

## Analysis

## Conclusion

## Strengths and Limitations